***Challenge 1***

Here the goal is to train on 25 samples. In this preliminary testbed the evaluation will be done on a 2000 sample validation set. Note in the end the final evaluation will be done on the full CIFAR-10 test set as well as potentially a separate dataset. The validation samples here should not be used for training in any way, the final evaluation will provide only random samples of 25 from a datasource that is not the CIFAR-10 training data. 

Feel free to modify this testbed to your liking, including the normalization transformations etc. Note however the final evaluation testbed will have a rigid set of components where you will need to place your answer. The only constraint is the data. Refer to the full project instructions for more information.


Setup training functions. Again you are free to fully modify this testbed in your prototyping within the constraints of the data used. You can use tools outside of pytorch for training models if desired as well although the torchvision dataloaders will still be useful for interacting with the cifar-10 dataset. 

In [ ]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
class CNNEncoder(nn.Module):
    """docstring for ClassName"""
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(3,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())
        self.layer4 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())

    def forward(self,x):
        if len(x.shape)<4:
          # print('TRUE')
          x  = x.unsqueeze(0)
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        #out = out.view(out.size(0),-1)
        return out # 64


class Net(torch.nn.Module):
    def __init__(self,input_size,hidden_size):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
                        # nn.Conv2d(64*2,64,kernel_size=3,padding=0),
                        nn.Conv2d(3,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        

        
# self.fc1 = nn.Linear(128*2*2,hidden_size)
        
      
        # self.fc1 = nn.Linear(input_size*3*3,hidden_size)
        # self.fc1 = nn.Linear(2304,hidden_size)
        # self.fc1 = nn.Linear(64,hidden_size)
        # self.fc1 = nn.Linear (10816,hidden_size)
        self.fc1 = nn.Linear(3136,hidden_size)

        self.fc2 = nn.Linear(hidden_size,10)

    def forward(self,x):
        if len(x.shape)<4:
          # print('TRUE')
          x  = x.unsqueeze(0)
        out = self.layer1(x)
        # print(f'dim layer 1 = {out.shape}')
        out = self.layer2(out)
        # print(f'dim layer 2 = {out.shape}')
#
        # out = self.layer3(out)
        # print(f'dim layer 3 = {out.shape}')
        
        out = out.view(out.size(0),-1)
        # print(f'dim flatten = {out.shape}')
        out = F.relu(self.fc1(out))
        # print(f'dim fc1 = {out.shape}')

        out = F.sigmoid(self.fc2(out))
        return out

class CNNEncoderNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CNNEncoderNet, self).__init__()
        self.cnn_encoder = CNNEncoder()
        self.net = Net(input_size, hidden_size)
        self.cnn_encoder.to(device)
        self.net.to(device)

    def forward(self, x):
        out = self.cnn_encoder(x)
        out = torch.cat((out, out), dim=1)  # concatenate the output of the CNN encoder
        out = self.net(out)
        return out

The below tries  2 random problem instances. In your development you may choose to prototype with 1 problem instances but keep in mind for small sample problems the variance is high so continously evaluating on several subsets will be important.

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset

  
from torchvision import datasets, transforms
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device = 'cpu'
##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    

accs = []

for seed in range(1, 15):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))
  
  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=64, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  
  input_size,hidden_size = 16,256
  model = Net(input_size,hidden_size)
  # model = CNNEncoderNet (input_size,hidden_size)
  model.to(device)
  optimizer = torch.optim.SGD(model.parameters(),lr=0.01, momentum=0.9, weight_decay=0.0005)
  # optimizer = torch.optim.Adam(model.parameters(),lr=0.005,)

  for epoch in range(100):
    train(model, device, train_loader, optimizer, epoch, display=epoch%5==0)
    
  accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 15 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.253584
Train Epoch: 5 [0/50 (0%)]	Loss: 1.815078
Train Epoch: 10 [0/50 (0%)]	Loss: 1.623162
Train Epoch: 15 [0/50 (0%)]	Loss: 1.600276
Train Epoch: 20 [0/50 (0%)]	Loss: 1.598444
Train Epoch: 25 [0/50 (0%)]	Loss: 1.598152
Train Epoch: 30 [0/50 (0%)]	Loss: 1.598074
Train Epoch: 35 [0/50 (0%)]	Loss: 1.598045
Train Epoch: 40 [0/50 (0%)]	Loss: 1.598031
Train Epoch: 45 [0/50 (0%)]	Loss: 1.598025
Train Epoch: 50 [0/50 (0%)]	Loss: 1.598021
Train Epoch: 55 [0/50 (0%)]	Loss: 1.598019
Train Epoch: 60 [0/50 (0%)]	Loss: 1.598017
Train Epoch: 65 [0/50 (0%)]	Loss: 1.598017
Train Epoch: 70 [0/50 (0%)]	Loss: 1.598016
Train Epoch: 75 [0/50 (0%)]	Loss: 1.598016
Train Epoch: 80 [0/50 (0%)]	Loss: 1.598015
Train Epoch: 85 [0/50 (0%)]	Loss: 1.598015
Train Epoch: 90 [0/50 (0%)]	Loss: 1.598015
Train Epoch: 95 [0/50 (0%)]	Loss: 1.598014

Test set: Ave

***Challenge 2***

You may use the same testbed but without the constraints on external datasets or models trained on exeternal datasets. You may not however use any of the CIFAR-10 training set. 

In [ ]:
import torchvision.models as models

from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset
import torch
import torch.nn as nn 
import torch.nn.functional as F
  
from torchvision import datasets, transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)

        # loss1 = F.cross_entropy(output[0], target)
        # loss2 = F.cross_entropy(output[1], target)
        # loss3 = F.cross_entropy(output[2], target)
        # loss = loss1+loss2+loss3
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

# We resize images to allow using imagenet pre-trained models, is there a better way?
resize = transforms.Resize(224) 

transform_val = transforms.Compose([resize, transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([resize, transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device) # you will really need gpu's for this part
    
##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api (lol)
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    
accs = []

for seed in range(1, 5):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 5000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))
  
  train_loader = torch.utils.data.DataLoader(train_data,
                                            #  batch_size=128, 
                                             batch_size = 64,  # For VGG
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                          #  batch_size=128, 
                                           batch_size = 64, # For VGG
                                           shuffle=False)
  
  model = CNNEncoderNet (16,128)

  # model = models.alexnet(pretrained=True)
  # model = models.resnet50(pretrained=True)
  # model = models.vgg19( ) # X
  # model = models.GoogLeNet()
  # model = models.vit_l_32(weights = 'IMAGENET1K_V1')


  # model.classifier = nn.Linear(256 * 6 * 6, 10) #alexnet
  
  # num_ftrs = model.fc.in_features
  # model.fc = nn.Linear(num_ftrs, 10) # resnet & GoogleNet

  # num_features = model.classifier[-1].in_features
  # model.classifier[-1] = nn.Linear(num_features, 10)


  optimizer = torch.optim.SGD(model.parameters(), 
                              lr=0.01, momentum=0.9,
                              weight_decay=0.0005)


  # optimizer = torch.optim.SGD(model.classifier.parameters(), 
  #                             lr=0.01, momentum=0.9,
  #                             weight_decay=0.0005)

  # optimizer = torch.optim.SGD(model.fc.parameters(), # for resnet Googlenet
  #                           lr=0.01, momentum=0.9,
  #                           weight_decay=0.0005)

  # optimizer = torch.optim.SGD(model.classifier[-1].parameters(), # for resnet Googlenet
  #                         lr=0.01, momentum=0.9,
  #                         weight_decay=0.0005)

  # optimizer = torch.optim.Adam(model.classifier.parameters())
  # optimizer = torch.optim.Adam(model.parameters())
  model.to(device)
  for epoch in range(10):
    train(model, device, train_loader, optimizer, epoch, display=True)
    
  accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))

cuda
Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
dim flatten = torch.Size([50, 10816])
Train Epoch: 0 [0/50 (0%)]	Loss: 2.347654
dim flatten = torch.Size([50, 10816])
Train Epoch: 1 [0/50 (0%)]	Loss: 2.181205
dim flatten = torch.Size([50, 10816])
Train Epoch: 2 [0/50 (0%)]	Loss: 1.983521
dim flatten = torch.Size([50, 10816])
Train Epoch: 3 [0/50 (0%)]	Loss: 1.834520
dim flatten = torch.Size([50, 10816])
Train Epoch: 4 [0/50 (0%)]	Loss: 1.743644
dim flatten = torch.Size([50, 10816])
Train Epoch: 5 [0/50 (0%)]	Loss: 1.680331
dim flatten = torch.Size([50, 10816])
Train Epoch: 6 [0/50 (0%)]	Loss: 1.636407
dim flatten = torch.Size([50, 10816])
Train Epoch: 7 [0/50 (0%)]	Loss: 1.613036
dim flatten = torch.Size([50, 10816])
Train Epoch: 8 [0/50 (0%)]	Loss: 1.603596
dim flatten = torch.Size([50, 10816])
Train Epoch: 9 [0/50 (0%)]	Loss: 1.600180
dim flatten = torch.Size([64, 10816])
dim flatten = torch.Size([64,